## 4.4.1. Przygotowanie danych do klasyfikacji

#### Generowanie plików pythonowych na podstawie notebooków

In [1]:
!jupyter nbconvert --to script copy_files_classification.ipynb
!jupyter nbconvert --to script split_data.ipynb

[NbConvertApp] Converting notebook copy_files_classification.ipynb to script
[NbConvertApp] Writing 1942 bytes to copy_files_classification.py
[NbConvertApp] Converting notebook split_data.ipynb to script
[NbConvertApp] Writing 2126 bytes to split_data.py


#### Importy

In [2]:
import tensorflow as tf
import numpy as np
import cv2
from pathlib import Path
from copy_files_classification import process_copy_classification
from split_data import run_data_split

#### Konfiguracja ścieżek


In [ ]:
input_dir = Path("./data/processed/images")
output_dir = Path("./data/processed/resized/resnet")
output_dir.mkdir(parents=True, exist_ok=True)

#### Załadowanie modelu U-Net

In [ ]:
model = tf.keras.models.load_model("./unet_best_model.h5", compile=False)

#### Funkcja `predict_mask`
Funkcja odpowiadająca za predykcje maski z modelu U-Net i jej przeskalowanie


In [ ]:
def predict_mask(image, model):
    resized_image = cv2.resize(image, (256, 256))
    normalized_image = resized_image / 255.0
    input_tensor = np.expand_dims(normalized_image, axis=0)
    predicted = model.predict(input_tensor)[0, :, :, 0]
    binary_mask = (predicted > 0.5).astype(np.uint8) * 255
    return cv2.resize(binary_mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)


#### Funkcja `get_padded_bbox`
Funkcja odpowiedzialna za znalezienie rozszerzonego bounding boxa na podstawie maski.

In [ ]:
def get_padded_bbox(mask, padding_ratio=0.1):
    kernel = np.ones((20, 20), np.uint8)
    dilated = cv2.dilate(mask, kernel, iterations=1)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None

    x_min, y_min = float('inf'), float('inf')
    x_max, y_max = 0, 0

    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x + w)
        y_max = max(y_max, y + h)

    w = x_max - x_min
    h = y_max - y_min
    x_min = max(int(x_min - w * padding_ratio), 0)
    y_min = max(int(y_min - h * padding_ratio), 0)
    x_max = min(int(x_max + w * padding_ratio), mask.shape[1])
    y_max = min(int(y_max + h * padding_ratio), mask.shape[0])

    return x_min, y_min, x_max, y_max


#### Funkcja `process_image`
Funkcja odpowiedzialna za przetworzenie (obrazu maskowanie, wycięcie, resize i zapis)

In [ ]:
def process_image(image_path, model, output_dir):
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Nie udało się wczytać: {image_path}")
        return

    mask = predict_mask(image, model)
    bbox = get_padded_bbox(mask)

    if bbox is None:
        print(f"Brak segmentacji dla {image_path}.")
        return

    x_min, y_min, x_max, y_max = bbox
    cropped = image[y_min:y_max, x_min:x_max]
    resized_cropped = cv2.resize(cropped, (224, 224))

    cv2.imwrite(str(output_dir / image_path.name), resized_cropped)


#### Przetwarzanie całego katalogu obrazów

In [ ]:
for image_path in input_dir.glob("*.[jp][pn]g"):
    process_image(image_path, model, output_dir)

#### Przeprowadzenie podziału danych na zbiór teningowy i testowy i zapisanie danych do plików .csv

In [3]:
run_data_split(Path("./data/processed/HAM10000_metadata_clean.csv"), Path("./data/processed/images_to_train/resnet"), 0.10, 42)

Zapisano train.csv i test.csv w data\processed\images_to_train\resnet
Wierszy: train=9007, test=1008


#### Skopiowanie zdjęć zawartych w pliku test.csv do folderu test 

In [4]:
process_copy_classification(Path("./data/processed/resized/resnet"), Path("./data/processed/images_to_train/resnet/test.csv"), Path("./data/processed/images_to_train/resnet"), "test")

Skopiowano 1008 plików z `resnet` do `data\processed\images_to_train\resnet\test`


#### Skopiowanie zdjęć zawartych w pliku train.csv do folderu train 

In [5]:
process_copy_classification(Path("./data/processed/resized/resnet"), Path("./data/processed/images_to_train/resnet/train.csv"), Path("./data/processed/images_to_train/resnet"), "train")

Skopiowano 9007 plików z `resnet` do `data\processed\images_to_train\resnet\train`
